In [1]:
# 'capture' magic prevents long outputs from spamming your notebook
#%%capture pipoutput

# For loading predefined environment variables from files
# Typically used to load sensitive access credentials
%pip install python-dotenv

# Standard python package for interacting with S3 buckets
%pip install boto3

# Interacting with Trino and using Trino with sqlalchemy
%pip install trino sqlalchemy sqlalchemy-trino

# Pandas and parquet file i/o
%pip install pandas pyarrow fastparquet

# OS-Climate utilities to make data ingest easier
%pip install osc-ingest-tools
%pip install ipython --upgrade
%pip install country_converter --upgrade
%pip install pint-pandas
%pip install openscm-units
%pip install pint
%pip install jedi
%pip install pycountry


You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the 

In [2]:
from dotenv import dotenv_values, load_dotenv
import osc_ingest_trino as osc
import os
import pathlib
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

In [3]:
import trino
from sqlalchemy.engine import create_engine

env_var_prefix = 'TRINO'

sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ[f'{env_var_prefix}_USER'],
    host = os.environ[f'{env_var_prefix}_HOST'],
    port = os.environ[f'{env_var_prefix}_PORT']
)
sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ[f'{env_var_prefix}_PASSWD']),
    'http_scheme': 'https',
    'catalog': 'osc_datacommons_dev'
}
engine = create_engine(sqlstring, connect_args = sqlargs)
connection = engine.connect()

trino_bucket = osc.attach_s3_bucket("S3_DEV")

/opt/app-root/lib64/python3.8/site-packages/trino/sqlalchemy/dialect.py:286: SAWarning: Dialect trino:rest will not make use of SQL compilation caching as it does not set the 'supports_statement_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Dialect maintainers should seek to set this attribute to True after appropriate development and testing for SQLAlchemy 1.4 caching support.   Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/14/cprf)
  res = connection.execute(sql.text(query))


In [4]:
import boto3

s3_source = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
)
source_bucket = s3_source.Bucket(os.environ['S3_LANDING_BUCKET'])

In [5]:
# define source and destination tables
# LULUCF (Land Use, Land-Use Change and Forestry)
ingest_catalog = 'osc_datacommons_dev'
ingest_schema = 'pcaf_sovereign_footprint'
ingest_table = 'sf_scope_results'
src_table_1 = 'sf_oecd_imgr_fco2'


In [6]:
def requantify_df(df):
    units_col = None
    columns_reversed = reversed(df.columns)
    for col in columns_reversed:
        if col.endswith("_units"):
            if units_col:
                # We expect _units column to follow a non-units column
                raise ValueError
            units_col = col
            continue
        if units_col:
            if col + '_units' != units_col:
                raise ValueError
            if (df[units_col]==df[units_col][0]).all():
                # Make a PintArray
                new_col = PintArray(df[col], dtype=f"pint[{ureg(df[units_col][0]).u}]")
            else:
                # Make a pd.Series of Quantity in a way that does not throw UnitStrippedWarning
                new_col = pd.Series(data=df[col], name=col) * pd.Series(data=df[units_col].map(lambda x: ureg(x).u), name=col)
            df = df.drop(columns=units_col)
            df[col] = new_col
            units_col = None
    return df


In [7]:
import pandas as pd
from functools import reduce
import matplotlib.pyplot as plot
import pandas as pd
import pint
from pint import set_application_registry, Quantity
from pint_pandas import PintArray, PintType
from pint_pandas.pint_array import is_pint_type
from openscm_units import unit_registry
#PintType.ureg = unit_registry
ureg = unit_registry
Q_ = ureg.Quantity
ureg.define("Tonnes=t=ton")
ureg.define("Millions=Million")
#ureg.define("USD=[currency]=$")
set_application_registry(ureg)


sql=f"""
select country_iso_code,partner_iso_code,industry_code,validity_date,attribute,value,value_units from {ingest_catalog}.{ingest_schema}.{src_table_1} """ 
df1 = pd.read_sql(sql, engine)
df1
#df1["units"] = "kt"
#df1 = requantify_df(df1).convert_dtypes()
#df1.info(verbose=True)
#df1


,country_iso_code,partner_iso_code,industry_code,validity_date,attribute,value,value_units
0,LTU,DNK,D35,2003,Foreign CO2 emissions embodied in gross imports,0.000,Millions Tonnes
1,LTU,DNK,D35,2004,Foreign CO2 emissions embodied in gross imports,0.000,Millions Tonnes
2,LTU,DNK,D35,2005,Foreign CO2 emissions embodied in gross imports,0.000,Millions Tonnes
3,LTU,DNK,D35,2006,Foreign CO2 emissions embodied in gross imports,0.000,Millions Tonnes
4,LTU,DNK,D35,2007,Foreign CO2 emissions embodied in gross imports,0.000,Millions Tonnes
...,...,...,...,...,...,...,...
334651,CAN,G20,D35,1998,Foreign CO2 emissions embodied in gross imports,19.475,Millions Tonnes
334652,CAN,G20,D35,1999,Foreign CO2 emissions embodied in gross imports,13.804,Millions Tonnes
334653,CAN,G20,D35,2000,Foreign CO2 emissions embodied in gross imports,13.593,Millions Tonnes
334654,CAN,G20,D35,2001,Foreign CO2 emissions embodied in gross imports,16.841,Millions Tonnes


<font size=3> Remove invalid country codes </font>

In [8]:
import pycountry
import pandas as pd
df_country = pd.DataFrame([country.__dict__['_fields'] for country in pycountry.countries])
df_country = df_country.rename(columns={'alpha_3':'country_iso_code'})
df_country=df_country[['country_iso_code']]
# add Rest of World to the country dataframe
dict_row = {'country_iso_code':'ROW'}
df_country = df_country.append(dict_row, ignore_index = True)
df_country.info(verbose=True)
df1=df1.convert_dtypes()
#df1.info(verbose=True)
df1 = pd.merge(df1,df_country,on=['country_iso_code'])  
df1=df1.convert_dtypes()
#df_result.drop(columns=['country_name_y'],inplace=True)
df1


/tmp/ipykernel_3828/2921960101.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_country = df_country.append(dict_row, ignore_index = True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 1 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   country_iso_code  250 non-null    object
dtypes: object(1)
memory usage: 2.1+ KB


,country_iso_code,partner_iso_code,industry_code,validity_date,attribute,value,value_units
0,LTU,DNK,D35,2003,Foreign CO2 emissions embodied in gross imports,0.0,Millions Tonnes
1,LTU,DNK,D35,2004,Foreign CO2 emissions embodied in gross imports,0.0,Millions Tonnes
2,LTU,DNK,D35,2005,Foreign CO2 emissions embodied in gross imports,0.0,Millions Tonnes
3,LTU,DNK,D35,2006,Foreign CO2 emissions embodied in gross imports,0.0,Millions Tonnes
4,LTU,DNK,D35,2007,Foreign CO2 emissions embodied in gross imports,0.0,Millions Tonnes
...,...,...,...,...,...,...,...
270139,PRT,MMR,D35,2014,Foreign CO2 emissions embodied in gross imports,0.0,Millions Tonnes
270140,PRT,MMR,D35,2015,Foreign CO2 emissions embodied in gross imports,0.0,Millions Tonnes
270141,PRT,MMR,D35,2016,Foreign CO2 emissions embodied in gross imports,0.0,Millions Tonnes
270142,PRT,MMR,D35,2017,Foreign CO2 emissions embodied in gross imports,0.0,Millions Tonnes


Calcuate Scope 2:
GHG emissions occurring as a consequence of the domestic use of grid-supplied electricity, heat,
steam and/or cooling which is imported from another territory


In [11]:
rename_columns = {'value':'total','partner_iso_code':'country_iso_code','value_units' :'total_units'}
agg_columns = { 'value' : 'sum'}
columns_order = ['attribute','industry_code','country_iso_code','validity_date','total','total_units']
df1 = df1.groupby(['partner_iso_code','industry_code','attribute','validity_date','value_units'],as_index=False).agg(agg_columns).rename(columns=rename_columns).reindex(columns=columns_order)

df1.convert_dtypes()

df1.info(verbose=True)
df1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4032 entries, 0 to 4031
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   attribute         4032 non-null   string 
 1   industry_code     4032 non-null   string 
 2   country_iso_code  4032 non-null   string 
 3   validity_date     4032 non-null   Int64  
 4   total             4032 non-null   Float64
 5   total_units       4032 non-null   string 
dtypes: Float64(1), Int64(1), string(4)
memory usage: 197.0 KB


,attribute,industry_code,country_iso_code,validity_date,total,total_units
0,Foreign CO2 emissions embodied in gross imports,D35,APEC,1995,39.478,Millions Tonnes
1,Foreign CO2 emissions embodied in gross imports,D35,APEC,1996,39.637,Millions Tonnes
2,Foreign CO2 emissions embodied in gross imports,D35,APEC,1997,46.108,Millions Tonnes
3,Foreign CO2 emissions embodied in gross imports,D35,APEC,1998,47.416,Millions Tonnes
4,Foreign CO2 emissions embodied in gross imports,D35,APEC,1999,47.128,Millions Tonnes
...,...,...,...,...,...,...
4027,Foreign CO2 emissions embodied in gross imports,DTOTAL,ZSCA,2014,209.262,Millions Tonnes
4028,Foreign CO2 emissions embodied in gross imports,DTOTAL,ZSCA,2015,214.719,Millions Tonnes
4029,Foreign CO2 emissions embodied in gross imports,DTOTAL,ZSCA,2016,206.428,Millions Tonnes
4030,Foreign CO2 emissions embodied in gross imports,DTOTAL,ZSCA,2017,209.894,Millions Tonnes


In [22]:
# divide the dataframe into two dataframes 
# to calculate Scope2 and Scope3 Values
df_D35 = df1[df1['industry_code']=='D35']
df_D35 = df_D35[['attribute','country_iso_code','validity_date','total','total_units']]
df_DTOTAL = df1[df1['industry_code']=='DTOTAL']
df_DTOTAL = df_DTOTAL[['country_iso_code','validity_date','total','total_units']]
df_result = pd.merge(df_D35,df_DTOTAL,on=['country_iso_code','validity_date'])  
df_result=df_result.convert_dtypes()
#df_result.drop(columns=['attribute_y','total_units_y'],inplace=True)
rename_columns = {'total_x':'scope2_value','total_y':'scope3_value','total_units_x':'scope2_value_units','total_units_y':'scope3_value_units'}
df_result.rename(columns=rename_columns,inplace=True)
df_result['scope3_value'] = df_result['scope3_value'] - df_result['scope2_value'] 
df_result[df_result['country_iso_code']=='CAN']

,attribute,country_iso_code,validity_date,scope2_value,scope2_value_units,scope3_value,scope3_value_units
216,Foreign CO2 emissions embodied in gross imports,CAN,1995,5.129,Millions Tonnes,179.111,Millions Tonnes
217,Foreign CO2 emissions embodied in gross imports,CAN,1996,5.269,Millions Tonnes,188.198,Millions Tonnes
218,Foreign CO2 emissions embodied in gross imports,CAN,1997,6.362,Millions Tonnes,202.972,Millions Tonnes
219,Foreign CO2 emissions embodied in gross imports,CAN,1998,7.993,Millions Tonnes,214.108,Millions Tonnes
220,Foreign CO2 emissions embodied in gross imports,CAN,1999,9.1,Millions Tonnes,222.868,Millions Tonnes
221,Foreign CO2 emissions embodied in gross imports,CAN,2000,18.191,Millions Tonnes,236.036,Millions Tonnes
222,Foreign CO2 emissions embodied in gross imports,CAN,2001,19.531,Millions Tonnes,228.887,Millions Tonnes
223,Foreign CO2 emissions embodied in gross imports,CAN,2002,8.195,Millions Tonnes,233.5,Millions Tonnes
224,Foreign CO2 emissions embodied in gross imports,CAN,2003,8.22,Millions Tonnes,232.392,Millions Tonnes
225,Foreign CO2 emissions embodied in gross imports,CAN,2004,8.411,Millions Tonnes,238.375,Millions Tonnes
